# Exploratory Data Analysis by SQL
<p> SQL techniques (Join, Cast, Aggregation and CTEs) are used in this notebook.</p>

## Key Findings
* There is a strong positive correlation between customer reviews and revenue. The company should start several initiatvies such as free sample and coupon to encourage customers to write comments more about their purchases.
<br />

* In terms of brand, the website currently sell 2575 Adidas products and 545 Nike products. Total revenue of Adidas (11.5k) is 14 times of Nike (0.8k). The better sales of Adidas list (avg. 4.5k per list) over Nike list (avg. 1.5k per list) is likely due to different price perception drove by different promotion strategy, considering Adidas products are on promotion by 33% discount and Nike products didn't run any promotion on the website.
<br />

* There are currently 3179 product listings on the site with 2928 products have traffic. The 251 long-tail selection without traffic could be removed from inventory to optimize free cash flow.
<br />

* 2,700 of the company's lists are footwear, accounting for 85% of the company's selection and generated an average revenue of $3,118 per product, 6 times that of clothing products.

In [13]:
%%sql
postgresql:///sports

SELECT COUNT(*) AS total_rows,
    COUNT(i.description) AS count_description,
    COUNT(f.listing_price) AS count_listing_price,
    COUNT(t.last_visited) AS count_last_visited
FROM info AS i
INNER JOIN finance AS f
    ON i.product_id = f.product_id
INNER JOIN traffic AS t
    ON t.product_id = f.product_id;

1 rows affected.


total_rows,count_description,count_listing_price,count_last_visited
3179,3117,3120,2928


In [17]:
%%sql

WITH footwear AS
(
    SELECT i.description, f.revenue
    FROM info AS i
    INNER JOIN finance AS f 
        ON i.product_id = f.product_id
    WHERE i.description ILIKE '%shoe%'
        OR i.description ILIKE '%trainer%'
        OR i.description ILIKE '%foot%'
        AND i.description IS NOT NULL
)

SELECT COUNT(*) AS num_footwear_products, 
    percentile_disc(0.5) WITHIN GROUP (ORDER BY revenue) AS average_footwear_revenue
FROM footwear;

 * postgresql:///sports
1 rows affected.


num_footwear_products,average_footwear_revenue
2700,3118.36


In [16]:
%%sql

WITH footwear AS
(
SELECT i.description, f.revenue
FROM info AS i
INNER JOIN finance AS f 
ON i.product_id = f.product_id
WHERE i.description ILIKE '%shoe%'
OR i.description ILIKE '%trainer%'
OR i.description ILIKE '%foot%'
AND i.description IS NOT NULL
)

SELECT COUNT(i.*) AS num_clothing_products, 
    percentile_disc(0.5) WITHIN GROUP (ORDER BY f.revenue) AS average_clothing_revenue
FROM info AS i
INNER JOIN finance AS f on i.product_id = f.product_id
WHERE i.description NOT IN (SELECT description FROM footwear);


 * postgresql:///sports
1 rows affected.


num_clothing_products,average_clothing_revenue
417,503.82


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert "total_rows" in set(last_output_df.columns), \
    """Did you alias the count of all products as "total_rows"?"""
    assert set(last_output_df.columns) == set(['total_rows', 'count_description', 'count_listing_price',
           'count_last_visited']), \
    """Did you select four columns and use the aliases in the instructions?"""

def test_shape():
    assert last_output_df.shape[0] == 1, \
    """Did you return a single row containing the count of values for each column?"""
    assert last_output_df.shape[1] == 4, \
    """Did you select four columns?"""

def test_values():
    assert last_output_df.values.tolist() == [[3179, 3117, 3120, 2928]], \
    """Did you correctly calculate the values for each column? Expected different results."""

## 2. Nike vs Adidas pricing
<p>We can see the database contains 3,179 products in total. Of the columns we previewed, only one &mdash; <code>last_visited</code> &mdash; is missing more than five percent of its values. Now let's turn our attention to pricing. </p>
<p>How do the price points of Nike and Adidas products differ? Answering this question can help us build a picture of the company's stock range and customer market. We will run a query to produce a distribution of the <code>listing_price</code> and the count for each price, grouped by <code>brand</code>. </p>

In [9]:
%%sql

SELECT b.brand, COUNT(f.*), CAST(SUM(f.revenue) as DECIMAL(18,0)) as total_revenue,
COUNT(f.*) / total_revenue as  
FROM finance AS f
INNER JOIN brands AS b 
ON f.product_id = b.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand
ORDER BY total_revenue DESC;

 * postgresql:///sports
2 rows affected.


brand,count,total_revenue
Adidas,2575,11526619
Nike,545,802283


In [5]:
%%sql

SELECT b.brand, f.listing_price::integer, COUNT(f.*)
FROM finance AS f
INNER JOIN brands AS b 
    ON f.product_id = b.product_id
WHERE listing_price > 0
GROUP BY b.brand, f.listing_price
ORDER BY listing_price DESC;

 * postgresql:///sports
77 rows affected.


brand,listing_price,count
Adidas,300,2
Adidas,280,4
Adidas,240,5
Adidas,230,8
Adidas,220,11
Nike,200,1
Adidas,200,8
Nike,190,2
Adidas,190,7
Nike,180,4


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'count', 'listing_price']), \
    """Did you select the correct columns and alias the first as "total_rows"?"""

def test_shape():
    assert last_output_df.shape[0] == 77, \
    """Did you correctly aggregate by brand? Expected the output to contain 77 products."""
    assert last_output_df.shape[1] == 3, \
    """The output should contain three columns: "brand", "listing_price", and "count"?"""

def test_values():
    assert last_output_df.iloc[0].values.tolist() == ['Adidas', 300, 2], \
    """Did you sort the results by "listing_price" in descending order?"""

## 3. Labeling price ranges
<p>It turns out there are 77 unique prices for the products in our database, which makes the output of our last query quite difficult to analyze. </p>
<p>Let's build on our previous query by assigning labels to different price ranges, grouping by <code>brand</code> and <code>label</code>. We will also include the total <code>revenue</code> for each price range and <code>brand</code>. </p>

In [10]:
%%sql

SELECT b.brand, COUNT(f.*), CAST(SUM(f.revenue) as DECIMAL(18,0)) as total_revenue,
CASE WHEN f.listing_price < 50 THEN 'Budget'
    WHEN f.listing_price >= 50 AND f.listing_price < 90 THEN 'Average'
    WHEN f.listing_price >= 90 AND f.listing_price < 150 THEN 'Expensive'
    ELSE 'Elite' END AS price_category
FROM finance AS f
INNER JOIN brands AS b 
    ON f.product_id = b.product_id
WHERE b.brand IS NOT NULL
GROUP BY b.brand, price_category
ORDER BY total_revenue DESC;

 * postgresql:///sports
8 rows affected.


brand,count,total_revenue,price_category
Adidas,1189,4852192,Average
Adidas,453,3290185,Expensive
Adidas,799,1800693,Budget
Adidas,134,1583550,Elite
Nike,366,600908,Budget
Nike,78,107632,Expensive
Nike,52,70136,Elite
Nike,49,23607,Average


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'price_category', 'count', 'total_revenue']), \
    """Did you select the correct columns? Expected "brand", "listing_price", "count", and "total_revenue"."""
    
def test_shape():
    assert last_output_df.shape[0] == 8, \
    "Did you group by brand and labels? Expected there to be eight rows."
    assert last_output_df.shape[1] == 4, \
    "Did you select four columns?"
    
def test_values():
    assert last_output_df[:4].values.tolist() == [['Adidas', 849, 4626980.069999999, 'Expensive'],
     ['Adidas', 1060, 3233661.060000001, 'Average'],
     ['Adidas', 307, 3014316.8299999987, 'Elite'],
     ['Adidas', 359, 651661.1200000002, 'Budget']], \
    "Did you correctly calculate values for Adidas products? Expected something different."
    assert last_output_df[4:].values.tolist() == [['Nike', 357, 595341.0199999992, 'Budget'],
     ['Nike', 82, 128475.59000000003, 'Elite'],
     ['Nike', 90, 71843.15000000004, 'Expensive'],
     ['Nike', 16, 6623.5, 'Average']], \
    "Did you correctly calculate values for Nike products? Expected something different."

## 4. Average discount by brand
<p>Interestingly, grouping products by brand and price range allows us to see that Adidas items generate more total revenue regardless of price category! Specifically, <code>"Elite"</code> Adidas products priced \$129 or more typically generate the highest revenue, so the company can potentially increase revenue by shifting their stock to have a larger proportion of these products!</p>
<p>Note we have been looking at <code>listing_price</code> so far. The <code>listing_price</code> may not be the price that the product is ultimately sold for. To understand <code>revenue</code> better, let's take a look at the <code>discount</code>, which is the percent reduction in the <code>listing_price</code> when the product is actually sold. We would like to know whether there is a difference in the amount of <code>discount</code> offered between brands, as this could be influencing <code>revenue</code>.</p>

In [12]:
%%sql
    
SELECT b.brand,CAST(AVG(f.discount) as DECIMAL(18,2)) * 100 AS average_discount
FROM brands AS b
INNER JOIN finance AS f 
    ON b.product_id = f.product_id
GROUP BY b.brand
HAVING b.brand IS NOT NULL
ORDER BY average_discount;

 * postgresql:///sports
2 rows affected.


brand,average_discount
Nike,0.00
Adidas,33.00


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'average_discount']), \
    """Did you select the correct columns? Expected "brand" and "average_discount"."""
    
def test_shape():
    assert last_output_df.shape[0] == 2, \
    "Did you group by brand? Expected two rows, one per brand."
    assert last_output_df.shape[1] == 2, \
    "Did you select two columns?"
    
def test_values():
    assert last_output_df.iloc[:, 1].values.tolist() == [0.0, 33.452427184465606], \
    "Did you correctly calculate the average discount for the two brands?"

## 5. Correlation between revenue and reviews
<p>Strangely, no <code>discount</code> is offered on Nike products! In comparison, not only do Adidas products generate the most revenue, but these products are also heavily discounted! </p>
<p>To improve revenue further, the company could try to reduce the amount of discount offered on Adidas products, and monitor sales volume to see if it remains stable. Alternatively, it could try offering a small discount on Nike products. This would reduce average revenue for these products, but may increase revenue overall if there is an increase in the volume of Nike products sold. </p>
<p>Now explore whether relationships exist between the columns in our database. We will check the strength and direction of a correlation between <code>revenue</code> and <code>reviews</code>. </p>

In [18]:
%%sql

SELECT CAST(CORR(r.reviews, f.revenue) as DECIMAL(18,2)) AS review_revenue_corr
FROM reviews AS r
INNER JOIN finance AS f 
ON r.product_id = f.product_id;

 * postgresql:///sports
1 rows affected.


review_revenue_corr
0.65


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['review_revenue_corr']), \
    """Did you calculate the correlation between reviews and revenue, aliasing as "review_revenue_corr"?"""
    
def test_shape():
    assert last_output_df.shape == (1, 1), \
    "Did you calculate the correlation between reviews and revenue?"
    
def test_values():
    assert last_output_df.values.tolist() == [[0.6518512283481301]], \
    "Did you correctly calculate how reviews correlates with revenue?"

## 6. Ratings and reviews by product description length
<p>Interestingly, there is a strong positive correlation between <code>revenue</code> and <code>reviews</code>. This means, potentially, if we can get more reviews on the company's website, it may increase sales of those items with a larger number of reviews. </p>
<p>Perhaps the length of a product's <code>description</code> might influence a product's <code>rating</code> and <code>reviews</code> &mdash; if so, the company can produce content guidelines for listing products on their website and test if this influences <code>revenue</code>. Let's check this out!</p>

In [7]:
%%sql

SELECT TRUNC(LENGTH(i.description), -2) AS description_length,
    ROUND(AVG(r.rating::numeric), 2) AS average_rating
FROM info AS i
INNER JOIN reviews AS r 
ON i.product_id = r.product_id
WHERE i.description IS NOT NULL
GROUP BY description_length
ORDER BY description_length;

 * postgresql:///sports
7 rows affected.


description_length,average_rating
0,1.87
100,3.21
200,3.27
300,3.29
400,3.32
500,3.12
600,3.65


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['description_length', 'average_rating']), \
    """Did you select the correct columns use the aliases "description_length" and "average_rating"?"""
    
def test_shape():
    assert last_output_df.shape[0] == 7, \
    """Did you create bins of 100 characters for "description_length"? Expected the output to contain seven rows."""
    assert last_output_df.shape[1] == 2, \
    "Expected the output to contain two columns."
    
def test_values():
    last_output_df = last_output.DataFrame().values.astype("float")
    assert last_output_df[0].tolist() == [0.0, 1.87], \
    """Did you sort the results by "description_length" in ascending order?"""
    assert last_output_df[-1].tolist() == [600.0, 3.65], \
    "Did you correctly calculate the results? Expected a different average rating for the largest description length bin."

## 7. Reviews by month and brand
<p>Unfortunately, there doesn't appear to be a clear pattern between the length of a product's <code>description</code> and its <code>rating</code>.</p>
<p>As we know a correlation exists between <code>reviews</code> and <code>revenue</code>, one approach the company could take is to run experiments with different sales processes encouraging more reviews from customers about their purchases, such as by offering a small discount on future purchases. </p>
<p>Let's take a look at the volume of <code>reviews</code> by month to see if there are any trends or gaps we can look to exploit.</p>

In [5]:
%%sql

SELECT b.brand, DATE_PART('month', t.last_visited) AS month, COUNT(r.*) AS num_reviews
FROM brands AS b
INNER JOIN traffic AS t 
    ON b.product_id = t.product_id
INNER JOIN reviews AS r 
    ON t.product_id = r.product_id
GROUP BY b.brand, month
HAVING b.brand IS NOT NULL
    AND DATE_PART('month', t.last_visited) IS NOT NULL
ORDER BY b.brand, month;

 * postgresql:///sports
24 rows affected.


brand,month,num_reviews,total_order
Adidas,1.0,253,253
Adidas,2.0,272,272
Adidas,3.0,269,269
Adidas,4.0,180,180
Adidas,5.0,172,172
Adidas,6.0,159,159
Adidas,7.0,170,170
Adidas,8.0,189,189
Adidas,9.0,181,181
Adidas,10.0,192,192


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['brand', 'month', 'num_reviews']), \
    """Did you select the correct columns? Expected "brand", "month", and "num_reviews"."""
    
def test_shape():
    assert last_output_df.shape[0] == 24, \
    "Did you group by brand and month?"
    assert last_output_df.shape[1] == 3, \
    "Did you select three columns?"
    
def test_values():
    assert last_output_df.iloc[0].values.tolist() == ['Adidas', 1.0, 253], \
    "Expected the first row to contain the number of reviews for Adidas products in January."
    assert last_output_df.iloc[-1].values.tolist() == ['Nike', 12.0, 35.0], \
    "Expected the last row to contain the number of reviews for Nike products in December."
    assert max(last_output_df["num_reviews"]) == 272, \
    "Did you correctly calculate the number of reviews? Expected the largest number of reviews to be 272 for Adidas products in February."

## 8. Footwear product performance
<p>Looks like product reviews are highest in the first quarter of the calendar year, so there is scope to run experiments aiming to increase the volume of reviews in the other nine months!</p>
<p>So far, we have been primarily analyzing Adidas vs Nike products. Now, let's switch our attention to the type of products being sold. As there are no labels for product type, we will create a Common Table Expression (CTE) that filters <code>description</code> for keywords, then use the results to find out how much of the company's stock consists of footwear products and the median <code>revenue</code> generated by these items.</p>

In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['num_footwear_products', 'median_footwear_revenue']), \
    "Did you select the correct columns and use the aliases specified in the instructions?"
    
def test_shape():
    assert last_output_df.shape[0] == 1, \
    "Expected the output to contain one row."
    assert last_output_df.shape[1] == 2, \
    "Expected the output to contain two columns."
    
def test_values():
    assert last_output_df.iloc[0,0] == 2700, \
    "Did you count the number of footwear products?"
    assert last_output_df.iloc[0,1] == 3118.36, \
    "Did you calculate the median revenue for footwear products?"

## 9. Clothing product performance
<p>Recall from the first task that we found there are 3,117 products without missing values for <code>description</code>. Of those, 2,700 are footwear products, which accounts for around 85% of the company's stock. They also generate a median revenue of over $3000 dollars!</p>
<p>This is interesting, but we have no point of reference for whether footwear's <code>median_revenue</code> is good or bad compared to other products. So, for our final task, let's examine how this differs to clothing products. We will re-use <code>footwear</code>, adding a filter afterward to count the number of products and <code>median_revenue</code> of products that are not in <code>footwear</code>.</p>

 * postgresql:///sports
1 rows affected.


num_clothing_products,median_clothing_revenue
417,503.82


In [0]:
%%nose

last_output = _
last_output_df = last_output.DataFrame()

def test_columns():
    assert set(last_output_df.columns) == set(['num_clothing_products', 'median_clothing_revenue']), \
    "Did you select the correct columns and use the aliases specified in the instructions?"
    
def test_shape():
    assert last_output_df.shape[0] == 1, \
    "Expected the output to contain one row."
    assert last_output_df.shape[1] == 2, \
    "Expected the output to contain two columns."
    
def test_values():
    assert last_output_df.iloc[0,0] == 417, \
    "Did you count the number of clothing products? Expected there to be 417 items."
    assert last_output_df.iloc[0,1] == 503.82, \
    "Did you calculate the median revenue for clothing products? Expected it to be $503.82."